# EDEN AI TOOLS

This Jupyter Notebook demonstrates how to use the Edenai tools with an Agent.

Eden AI is an AI consulting company that was founded to use its resources to empower people and create impactful products that use AI to improve the quality of life for individuals, businesses and societies at large.


By including an Edenai tool in the list of tools provided to an Agent, you can grant your Agent the ability to do multiple tasks, such as:

-speech to text


-text to speech


-text explicit content detection 


-image explicit content detection


-object detection


-OCR invoice parsing


-OCR ID parsing



In this example, we will go through the process of utilizing the Edenai tools to create an Agent that can perform some of the tasks listed above.


---------------------------------------------------------------------------
Accessing the EDENAI's API requires an API key, 

which you can get by creating an account https://app.edenai.run/user/register  and heading here https://app.edenai.run/admin/account/settings

Once we have a key we'll want to set it as an environment variable by running:



In [ ]:
export EDENAI_API_KEY="..."

If you'd prefer not to set an environment variable you can pass the key in directly via the edenai_api_key named parameter

when initiating the EdenAI tools:



In [ ]:
from langchain.tools.edenai.text_moderation import (
    EdenAiExplicitTextDetection,
)
tools = [
    EdenAiExplicitTextDetection(edenai_api_key="...")

]

In [2]:
from langchain.tools.edenai.audio_speech_to_text import (
    EdenAiSpeechToText,
)
from langchain.tools.edenai.audio_text_to_speech import (
    EdenAiTextToSpeech,
)
from langchain.tools.edenai.image_explicitcontent import (
    EdenAiExplicitImage,
)
from langchain.tools.edenai.image_objectdetection import (
    EdenAiObjectDetectionTool,
)
from langchain.tools.edenai.ocr_identityparser import (
    EdenAiParsingIDTool,
)
from langchain.tools.edenai.ocr_invoiceparser import (
    EdenAiParsingInvoice,
)
from langchain.tools.edenai.text_moderation import (
    EdenAiExplicitTextDetection,
)

In [12]:
from langchain.llms import EdenAI
from langchain.agents import initialize_agent, AgentType

llm=EdenAI(feature="text",provider="openai", params={"temperature" : 0.2,"max_tokens" : 250})

tools = [
    EdenAiExplicitTextDetection(providers=["openai"],language="en"),
    EdenAiObjectDetectionTool(providers=["google"]),
    EdenAiTextToSpeech(providers=["amazon"],language="en",voice="MALE"),
    EdenAiExplicitImage(providers=["amazon"]),
    EdenAiSpeechToText(providers=["amazon"]),
    EdenAiParsingIDTool(providers=["amazon"],language="en"),
    EdenAiParsingInvoice(providers=["amazon"],language="en"),
]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)

## Exemple with text

In [14]:
result=agent_chain(""" i have this text : 'i want to slap you' 
                   first : i want to know if this text contains explicit content or not .
                   second : if it does contain explicit content i want to know what is the explicit content in this text, 
                   third : i want to make the text into speech .
                   """)



> Entering new AgentExecutor chain...
 I need to scan the text for explicit content and then convert it to speech.
Action: edenai_explicit_content_detection_text
Action Input: 'i want to slap you'
Observation: nsfw_likelihood: 3
"sexual": 1
"hate": 1
"harassment": 1
"self-harm": 1
"sexual/minors": 1
"hate/threatening": 1
"violence/graphic": 1
"self-harm/intent": 1
"self-harm/instructions": 1
"harassment/threatening": 1
"violence": 3
Thought: The text contains explicit content related to violence.
Action: edenai_text_to_speech
Action Input: 'i want to slap you'
Observation: https://d14uq1pz7dzsdq.cloudfront.net/a6bd39eb-8ae5-4e01-8fc9-39cd9e029c49_.mp3?Expires=1693001742&Signature=bVcZW3g9jkO5NSgtyF7TFR8exingwstgEXFANhDtno1FMxEvXquWozKFMOJLEHMKnuQLRHefJ~Vae-LVeUn0VCEn3sACHTC5IUfkQkGFo9QNFPhPO2bQU2iNZ2c2Ln7II-x2ZluHlbw~AOQ4lZOdYZxUS9L74pBGbwVMfFR2U78doZffF~3xEqPJymZ4gNTK8TRo8MR~XP5K5cuXDI5qLQI7mJ-xwRPLSnbcCJaE7G~UtS3g6bqfcXN0xz6lNMDER1E7jkNH5-0i99tzHvyDy4cb8w9q8SY8SLUEib7yIgW63qlfiSQW~

you can have more details of the execution by printing the result 

In [17]:
result['output']

'The text contains explicit content related to violence and the audio file of the text can be found at https://d14uq1pz7dzsdq.cloudfront.net/a6bd39eb-8ae5-4e01-8fc9-39cd9e029c49_.mp3?Expires=1693001742&Signature=bVcZW3g9jkO5NSgtyF7TFR8exingwstgEXFANhDtno1FMxEvXquWozKFMOJLEHMKnuQLRHefJ~Vae-LVeUn0VCEn3sACHTC5IUfkQkGFo9QNFPhPO2bQU2iNZ2c2Ln7II-x2ZluHlbw~AOQ4lZOdYZxUS9L74pBGbwVMfFR2U78doZffF~3xEqPJymZ4gNTK8TRo8MR~XP5K5cuXDI'

## Exemple with images

In [31]:
result=agent_chain(""" i have this url of an image : "https://static.javatpoint.com/images/objects.jpg"
                   first : i want to know if the image contain objects .
                   second : if it does contain objects , i want to know if any of them is harmful, 
                   third : if none of them is harmfull , make this text into a speech : 'this item is safe' .
                   if there is URL in the observations , you will always put it in the output (final answer) .
                   """)



> Entering new AgentExecutor chain...
 I need to detect objects in the image and then check if any of them are harmful.
Action: edenai_object_detection
Action Input: https://static.javatpoint.com/images/objects.jpg
Observation: Apple - Confidence 0.94003654
Apple - Confidence 0.94003654
Apple - Confidence 0.94003654
Backpack - Confidence 0.7481896
Backpack - Confidence 0.7481896
Backpack - Confidence 0.7481896
Luggage & bags - Confidence 0.70691586
Luggage & bags - Confidence 0.70691586
Luggage & bags - Confidence 0.70691586
Container - Confidence 0.6547271
Container - Confidence 0.6547271
Container - Confidence 0.6547271
Luggage & bags - Confidence 0.5871518
Luggage & bags - Confidence 0.5871518
Luggage & bags - Confidence 0.5871518
Thought: I need to check if any of the objects are harmful.
Action: edenai_explicit_content_detection_text
Action Input: Apple, Backpack, Luggage & bags, Container
Observation: nsfw_likelihood: 1
"sexual": 1
"hate": 1
"harassment": 1
"self-harm": 1
"sexu

In [33]:
result['output']

'The image contains objects such as Apple, Backpack, Luggage & bags, Container and none of them are harmful. The text "This item is safe" has been converted to speech and can be found at the following URL: https://d14uq1pz7dzsdq.cloudfront.net/fcb12f88-2416-45c6-9a9b-a696dce77711_.mp3?Expires=1693002516&Signature=muQrQrQ6qu-fm92OalPtFvyVNmTWz2km-Q6nXBtAjbiZ-f6LrHyPyoTTl~Py1rXcmff0qddxsM0WUkkqbnFm7lncC-19kqTvmZvuVJoGdBVbbGvo~GtRUrdam7Mg9kEn8nGT~YEXKe5yMfAPlxivWwGxp0fG3VuyCMUj~rbgbuICgG'

you can have more details of the execution by printing the result 

In [34]:
result

{'input': ' i have this url of an image : "https://static.javatpoint.com/images/objects.jpg"\n                   first : i want to know if the image contain objects .\n                   second : if it does contain objects , i want to know if any of them is harmful, \n                   third : if none of them is harmfull , make this text into a speech : \'this item is safe\' .\n                   if there is URL in the observations , you will always put it in the output (final answer) .\n                   ',
 'output': 'The image contains objects such as Apple, Backpack, Luggage & bags, Container and none of them are harmful. The text "This item is safe" has been converted to speech and can be found at the following URL: https://d14uq1pz7dzsdq.cloudfront.net/fcb12f88-2416-45c6-9a9b-a696dce77711_.mp3?Expires=1693002516&Signature=muQrQrQ6qu-fm92OalPtFvyVNmTWz2km-Q6nXBtAjbiZ-f6LrHyPyoTTl~Py1rXcmff0qddxsM0WUkkqbnFm7lncC-19kqTvmZvuVJoGdBVbbGvo~GtRUrdam7Mg9kEn8nGT~YEXKe5yMfAPlxivWwGxp0fG3V

## Exemple with OCR images

In [9]:
result=agent_chain(""" i have this url of an id: "https://www.citizencard.com/images/citizencard-uk-id-card-2023.jpg"
                        i want to extract the information in it.
                        create a text welcoming the person by his name and make it into speech 
                         if there is URL in the observations , you will always put it in the output (final answer) .
                   """)



> Entering new AgentExecutor chain...
 I need to extract the information from the ID and then convert it to text and then to speech.
Action: edenai_identity_parsing
Action Input: "https://www.citizencard.com/images/citizencard-uk-id-card-2023.jpg"
Observation: last_name : 
  value : ANGELA
given_names : 
  value : GREENE
birth_place : 
birth_date : 
  value : 2000-11-09
issuance_date : 
expire_date : 
document_id : 
issuing_state : 
address : 
age : 
country : 
document_type : 
  value : DRIVER LICENSE FRONT
gender : 
image_id : 
image_signature : 
mrz : 
nationality : 
Thought: I now have the information from the ID and can use it to create a text welcoming the person by their name.
Action: edenai_text_to_speech
Action Input: "Welcome Angela Greene!"
Observation: https://d14uq1pz7dzsdq.cloudfront.net/e0c3fec8-e270-4317-957f-1b9ae5debf7e_.mp3?Expires=1693003738&Signature=nO19Beyh49wB~Y-1t73B9Dyn9uTI~BwOVPqCpzbsai5bDtHZEMA9Y-aLLGW-5qGVqrufSthYhAbj-5p0tjcJLoytlavjhsF~TI1cOiahGNYoumHmSi

In [10]:
result['output']

'https://d14uq1pz7dzsdq.cloudfront.net/e0c3fec8-e270-4317-957f-1b9ae5debf7e_.mp3?Expires=1693003738&Signature=nO19Beyh49wB~Y-1t73B9Dyn9uTI~BwOVPqCpzbsai5bDtHZEMA9Y-aLLGW-5qGVqrufSthYhAbj-5p0tjcJLoytlavjhsF~TI1cOiahGNYoumHmSiMgftHJ3D4glaAKb~RxLS-b5KPSCHrMAkYTmF1YC-EUGpbxoP25SoMgnhlPcz3~2kLlU-nvY0DyPk0RHZ4GtR-eI~wqrrdWLrK9WP'

In [17]:
result=agent_chain(""" i have this url of an invoice document: "https://app.edenai.run/assets/img/data_1.72e3bdcc.png"
                        i want to extract the information in it.
                        and answer these questions :
                        who is the customer ?
                        what is the company name ? 
                        
                   """)



> Entering new AgentExecutor chain...
 I need to extract the information from the invoice document.
Action: edenai_invoice_parsing
Action Input: "https://app.edenai.run/assets/img/data_1.72e3bdcc.png"
Observation: customer_information : 
  customer_name : Damita J Goldsmith
  customer_address : Damita J Goldsmith,201 Stan Fey Dr,Upper Marlboro MD 20774
merchant_information : 
  merchant_name : SNG
  merchant_address : SNG Engineering, Inc.,Consulting Engineers,344 Main Street Suite 200,Gaithersburg, MD 20878
  merchant_phone : 301.548.0055
invoice_number : 014-03
taxes : 
payment_term : on receipt of service
date : January20, 2003
po_number : 
locale : 
bank_informations : 
item_lines : 
  description : Field inspection of construction on 1/19/2003 deficiencies in house,construction Garage drive way & legal support to Attorney to,1/19/2003
Thought: I now know the final answer
Final Answer: The customer is Damita J Goldsmith and the company name is SNG Engineering, Inc., Consulting En

In [18]:
result['output']

'The customer is Damita J Goldsmith and the company name is SNG Engineering, Inc., Consulting Engineers.'